# IMPORTAÇÕES DE PACOTES NECESSÁRIOS

In [ ]:
from keras.utils import HDF5Matrix
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
import keras.models 
import numpy as np
from keras.models import Sequential
from keras.models import model_from_json
import tensorflow as tf
from keras.layers import Dense
import matplotlib as plt
from keras.utils import np_utils
import os
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Input, Flatten,Conv2D, MaxPooling2D,Dropout,BatchNormalization,Embedding
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


# CARREGANDO BASE DE DADOS EM FORMATO HDF5Matrix

In [ ]:
X_Treino =np.asarray(HDF5Matrix('C:\\Users\\Renesio\\Documents\\TCC2\\camelyonpatch_level_2_split_train_x.h5~\\camelyonpatch_level_2_split_train_x.h5', 'x'))
Y_Treino =np.asarray(HDF5Matrix('C:\\Users\\Renesio\\Documents\\TCC2\\camelyonpatch_level_2_split_train_y.h5\\camelyonpatch_level_2_split_train_y.h5', 'y'))

datagen = ImageDataGenerator(
              preprocessing_function=lambda X_Treino: X_Treino/255.,
              width_shift_range=4,  # randomly shift images horizontally
              height_shift_range=4,  # randomly shift images vertically 
              horizontal_flip=True,  # randomly flip images
              vertical_flip=True)  # randomly flip images
              

# APLICANDO UM REDIMENSIONAMENTO NA MATRIZ DE LABELS 

In [ ]:
import tensorflow as tf
Y_Treino = Y_Treino.reshape(len(Y_Treino),1)


# CRIAÇÃO DO MODELO CONVOLUCIONAL A SER TREINADO 

In [ ]:
import os
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Input, Flatten,Conv2D, MaxPooling2D,Dropout,BatchNormalization

batch_size = 8


model = Sequential()

model.add(Conv2D(96, (3, 3), activation='relu', input_shape=(96,96,3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))



model.add(Flatten())

model.add(Dense(256, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))


In [ ]:
 model.summary()

# ADCIONANDO TIPOS DE OPTIMIZAÇÃO E METRICAS PARA O TREINO

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# TREINANDO O MODELO

In [ ]:
Hi = model.fit_generator(datagen.flow(X_Treino, Y_Treino, batch_size=4),
                    steps_per_epoch=len(X_Treino)/batch_size,
                    epochs=3, verbose = 1)

# SALVANDO MODELO TREINADO 

In [ ]:
from keras.models import model_from_json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    
    json_file.write(model_json)
    
model.save_weights("model.h5")
print("Saved model to disk")

# CAREGANDO BASE DE TESTES

In [ ]:
X_Teste =np.asarray(HDF5Matrix('C:\\Users\\Renesio\\Documents\\TCC2\\camelyonpatch_level_2_split_test_x.h5\\camelyonpatch_level_2_split_test_x.h5', 'x'))
Y_Teste = np.asarray(HDF5Matrix('C:\\Users\\Renesio\\Documents\\TCC2\\camelyonpatch_level_2_split_test_y.h5\\camelyonpatch_level_2_split_test_y.h5', 'y'))

datagen2= ImageDataGenerator(
              preprocessing_function=lambda x: x/255.,
              width_shift_range=4,  # randomly shift images horizontally
              height_shift_range=4,  # randomly shift images vertically 
              horizontal_flip=True,  # randomly flip images
              vertical_flip=True)  # randomly flip images
              

# REDIMENSIONANDO A MATRIZ DE LABELS

In [ ]:
Y_Teste = Y_Teste.reshape(len(Y_Teste),1)

In [ ]:
batch_size = 8

# CARREGANDO O MODELO TREINADO E TESTANDO A ACURACIA DO MODELO

In [ ]:
from keras.models import model_from_json

json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
score = loaded_model.evaluate_generator(datagen2.flow(XTeste3, YTeste1, batch_size=8),
                    steps=1000,
                    verbose = 1)

print("%s: %.2f%%" % (model.metrics_names[1], [1]*100))


# TRANFORMANDO OS DADOS DA MATRIZ DE TESTE 'X' PARA FLOAT .255

In [ ]:
XTeste= Xt.astype('float32')

XTeste /= 255

# PREDIZENDO RESULTADOS COM BASE NO TREINO NA BASE DE TESTES

In [ ]:
predicaoteste = model.predict_proba(X_Teste)

# RESULTADOS OBTIDOS COM O TESTE METRICAS( AUC, ACC, ESP, SENSS, KAPPA)

In [ ]:
from sklearn import metrics

import matplotlib.pyplot as plt

prediction = predicaoteste

fpr, tpr, thresholds = metrics.roc_curve(YTeste1,predicaoteste[:,1])
auc=metrics.auc(fpr, tpr)

fig = plt.figure()
fig.set_size_inches( 15, 5 )
rocCurve = fig.add_subplot( 1, 2, 1 )

rocCurve.plot(fpr, tpr, color = 'darkgreen', lw = 2, label = 'ROC curve (area = %0.2f)' % auc )
rocCurve.plot( [0, 1], [0, 1], color = 'navy', lw = 1, linestyle = '--' )
rocCurve.grid()
plt.xlim( [0.0, 1.0] )
rocCurve.set_xticks( np.arange( -0.1, 1.0, 0.1 ) )
plt.ylim( [0.0, 1.05] )
rocCurve.set_yticks( np.arange( 0, 1.05, 0.1 ) )
plt.xlabel( 'False Positive Rate' )
plt.ylabel( 'True Positive Rate' )
plt.title( 'ROC' )
rocCurve.legend( loc = "lower right" )

plt.show()

cm = metrics.confusion_matrix(YTeste1,predicaoteste[:,1].round(), labels=None, sample_weight=None)
TP = cm[0][0] 
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]

print("Matrix Confusion:\n\t- Verdadeiro Positivo: ", TP, "\n\t- Falso Positivo: ", FP, "\n\t- Falso Negativo: ", FN, "\n\t- Verdadeiro Negativo: ", TN )

total1=sum(sum(cm))

accuracy1=((TP+TN)/total1)
print ('Accuracy:\t', accuracy1)

sensitivity1 = TP/(TP+FP)
print('Sensitivity:\t', sensitivity1 )

specificity1 = TN/(FN+TN)
print('Specificity:\t', specificity1)

print("Kappa:\t\t", metrics.cohen_kappa_score(YTeste1,predicaoteste[:,1].round() ,labels=None, weights=None, sample_weight=None))

# SALVANDO RESULTADOS EM UM CSV

In [ ]:
import csv
resultado = []
with open('Result2.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    for i in range(len(X_Teste)):
    
        resultado.append([(i,predicaoteste[:,1][i])])
    
        print("[ %s ], Predicted=%s" % (i, predicaoteste[:,1][i]))
        spamwriter.writerow([[i] ,  predicaoteste[:,1][i]])
        